#Task one

You can find the dataset here.
Try with several hyperparameters values (maxIter and regParam).
For example, you can try maxIter = 5, 10, or 20. regParam = 0.1, 0.5, or 1.0
Make a conclusion which models performs better according to the RMSE.
You can put this conclusion inside your Jupyter notebook.


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

lines = spark.read.text("/content/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [4]:
iter = [5, 10, 20]
reg = [0.1, 0.5, 1.0]

for i in iter:
    for r in reg:
        als = ALS(maxIter=i, regParam=r, userCol="userId", itemCol="movieId", ratingCol="rating",
                coldStartStrategy="drop")
        model = als.fit(training)

        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)
        print(f"Root-mean-square error for maxIter = {i}, regParam = {r}: " + str(rmse))

Root-mean-square error for maxIter = 5, regParam = 0.1: 1.025699297680682
Root-mean-square error for maxIter = 5, regParam = 0.5: 1.2542259753109646
Root-mean-square error for maxIter = 5, regParam = 1.0: 1.5070491305752927
Root-mean-square error for maxIter = 10, regParam = 0.1: 0.9954649301428903
Root-mean-square error for maxIter = 10, regParam = 0.5: 1.251753933460711
Root-mean-square error for maxIter = 10, regParam = 1.0: 1.5070408409467195
Root-mean-square error for maxIter = 20, regParam = 0.1: 0.9907019112592589
Root-mean-square error for maxIter = 20, regParam = 0.5: 1.2517109148091836
Root-mean-square error for maxIter = 20, regParam = 1.0: 1.5070408409789606


The best combination of hyper parameters is maxIter = 20, regParam = 0.1 with ems = 0.991